In [1]:
import numpy as np
import pandas as pd
from textblob import TextBlob
from collections import defaultdict
from collections import Counter
import copy

In [2]:
dir ='/root/JupyterNotebook/kaggle-Redefining-Cancer-Treatment'

In [5]:
train_cleantxt = pd.read_csv(dir+'/5 middle for stage2/train_cleantxt.txt')
train_genevar = pd.read_csv(dir+'/4 input for stage2/training_variants.csv')


In [6]:
train_classtxt = pd.concat([train_cleantxt,train_genevar['Class']],axis=1)
train_classtxt.tail()

,ID,CleanText,Class
3684,15600,systematic characterization somatic mutation c...,4
3685,15609,von hippel-lindau tumor suppressor protein pvh...,4
3686,15626,genetic alteration activate mitogen-activated ...,6
3687,15628,abstract functional consequence missense varia...,4
3688,15650,transcription factor tumor suppressor protein ...,1


In [7]:

new_word_freq_allclass=[]
for i in range(9):
    t = i+1
    classdoc_df = train_classtxt['CleanText'][train_classtxt['Class']== t]
    word_freq_list=[]
    doc_number = len(classdoc_df)
    for doc in classdoc_df:
        word_freq = defaultdict(int)
        doc = TextBlob(doc)
        words = doc.words
        for word in words:
            word_freq[word] += 1
        word_freq_list.append(word_freq)
        
    word_freq_all = Counter()
    for index,word_f in enumerate(word_freq_list):
        if index == 0:
            word_freq_all = Counter(word_f)
        else:
            word_freq_all += Counter(word_f)
            
    new_word_freq_all = copy.deepcopy(word_freq_all) 
    new_word_freq_list=[]
    for word_freq in word_freq_list:
        new_word_freq = defaultdict(int)
        for word,count in word_freq.items():
            if word_freq_all[word] <=  3*count or word_freq_all[word] <= doc_number/5 :
                del new_word_freq_all[word]
            else:
                new_word_freq[word] = count
        new_word_freq_list.append(new_word_freq)
    
    print(len(new_word_freq_all)) 
    new_word_freq_allclass.append(new_word_freq_all)

word_freq_allclass = Counter()
for index,word_f in enumerate(new_word_freq_allclass):
        if index == 0:
            word_freq_allclass = Counter(word_f)
        else:
            word_freq_allclass += Counter(word_f)

print(len(word_freq_allclass))

2856
2784
2190
2895
2340
2383
3293
1500
2964
6562


In [8]:
import pickle
pickle.dump(word_freq_allclass, open(dir+'/5 middle for stage2/'+'word_freq_all_data', 'wb'))